In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVR
#from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
#from catboost import CatBoostRegressor
#from xgboost import XGBRegressor 
import warnings
import logger
from dataclasses import dataclass
from utils import save_object,evaluate_models
from exception import CustomException
import os
import sys


In [17]:
df=pd.read_csv("../data_and_eda/data/alzheimers_disease_data.csv")


In [18]:
df=df.drop('PatientID',axis=1)

In [19]:
df=df.drop('DoctorInCharge',axis=1)

In [20]:
x=df.drop('Diagnosis',axis=1)
y=df['Diagnosis']
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=0)


In [ ]:
logging.info()

In [21]:
model1=DecisionTreeClassifier()
model1.fit(xtrain,ytrain)


DecisionTreeClassifier()

In [22]:
predicted=model1.predict(xtest)
r2_square=r2_score(ytest,predicted)

In [23]:
r2_square

0.6049808199472548

In [24]:
model2=AdaBoostClassifier()
model2.fit(xtrain,ytrain)
predicted1=model2.predict(xtest)
r2_square1=r2_score(ytest,predicted1)
print(r2_square1)

0.7017202109805802


In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
model3=RandomForestClassifier(bootstrap= False, max_depth= 20, max_features='log2', min_samples_leaf= 1, min_samples_split= 10, n_estimators= 200)
model3.fit(xtrain,ytrain)
predicted2=model3.predict(xtest)
accuracy_score1=accuracy_score(ytest,predicted2)
print(accuracy_score1)

0.9684014869888475


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(xtrain, ytrain)

# Print the best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best parameters found:  {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Best accuracy:  0.9379189662134877


In [27]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [28]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU,PReLU,ELU,ReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [29]:
classifier=Sequential()
classifier.add(Dense(units=6,activation='relu'))
classifier.add(Dense(units=12,activation='relu'))
classifier.add(Dense(units=6,activation='relu'))
classifier.add(Dense(units=12,activation='relu'))
classifier.add(Dense(units=6,activation='relu'))
classifier.add(Dense(1,activation='sigmoid'))

classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) #adam uses a learning rate of 0.01

In [30]:
model_history=classifier.fit(xtrain,ytrain,validation_split=0.33,batch_size=10,epochs=360)

Epoch 1/360
108/108 [==============================] - 1s 3ms/step - loss: 1.2192 - accuracy: 0.5459 - val_loss: 0.6680 - val_accuracy: 0.5977
Epoch 2/360
108/108 [==============================] - 0s 1ms/step - loss: 0.6670 - accuracy: 0.6061 - val_loss: 0.6591 - val_accuracy: 0.6203
Epoch 3/360
108/108 [==============================] - 0s 1ms/step - loss: 0.6640 - accuracy: 0.6200 - val_loss: 0.6666 - val_accuracy: 0.5846
Epoch 4/360
108/108 [==============================] - 0s 1ms/step - loss: 0.6619 - accuracy: 0.6219 - val_loss: 0.6786 - val_accuracy: 0.5752
Epoch 5/360
108/108 [==============================] - 0s 1ms/step - loss: 0.6598 - accuracy: 0.6154 - val_loss: 0.6524 - val_accuracy: 0.6541
Epoch 6/360
108/108 [==============================] - 0s 1ms/step - loss: 0.6504 - accuracy: 0.6367 - val_loss: 0.6619 - val_accuracy: 0.6353
Epoch 7/360
108/108 [==============================] - 0s 1ms/step - loss: 0.6642 - accuracy: 0.6358 - val_loss: 0.6620 - val_accuracy: 0.6316

In [31]:
predicted3=classifier.predict(xtest)
predicted_classes = (predicted3 > 0.5).astype("int32")
accuracy = accuracy_score(ytest, predicted_classes)
print(f'Accuracy: {accuracy}')


17/17 [==============================] - 0s 720us/step
Accuracy: 0.8197026022304833


In [32]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        2149 non-null   int64  
 1   Gender                     2149 non-null   int64  
 2   Ethnicity                  2149 non-null   int64  
 3   EducationLevel             2149 non-null   int64  
 4   BMI                        2149 non-null   float64
 5   Smoking                    2149 non-null   int64  
 6   AlcoholConsumption         2149 non-null   float64
 7   PhysicalActivity           2149 non-null   float64
 8   DietQuality                2149 non-null   float64
 9   SleepQuality               2149 non-null   float64
 10  FamilyHistoryAlzheimers    2149 non-null   int64  
 11  CardiovascularDisease      2149 non-null   int64  
 12  Diabetes                   2149 non-null   int64  
 13  Depression                 2149 non-null   int64

In [33]:
df['Diagnosis']

0       0
1       0
2       0
3       0
4       0
       ..
2144    1
2145    1
2146    1
2147    1
2148    0
Name: Diagnosis, Length: 2149, dtype: int64